In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [ ]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [ ]:
#links
estados = ['https://www.conicyt.cl/blog/category/concursos/abiertos/', 'https://www.conicyt.cl/blog/category/concursos/en-evaluacion/', 'https://www.conicyt.cl/blog/category/concursos/adjudicados/', 'https://www.conicyt.cl/blog/category/concursos/proximos/']


def links_proyectos(estados_chile):
    est = pd.DataFrame()
    links = []
    cant_links_est = []
    for estado in estados:
        
        parserT = parser(estado)

        #Extrae y organiza lista de paginacion
        paginacion = parserT.xpath('//div[@class="pagenavi"]/ul//a/@href')
        
        urls = []
        pagina = estado
        if len(paginacion) != 0:
            paginacion.pop()
            paginacion.append('?pag=1')
            for link in paginacion:
                urls.append(pagina+link)
        else:
            urls.append(estado)
        

        

        #Extrae links de cada pag
        n = 0
        for url in urls:
            parser_url = parser(url)
            links_pagina = parser_url.xpath('//div[@id="contenido_doble"]//h2/a/@href')
            links += links_pagina
            n += len(links_pagina)


        cant_links_est.append(n)

    estados_proy = []
    for i in range(cant_links_est[0]):
        estados_proy.append('Abierto')
    for i in range(cant_links_est[1]):
        estados_proy.append('En evaluación')
    for i in range(cant_links_est[2]):
        estados_proy.append('Cerrados')
    for i in range(cant_links_est[3]):
        estados_proy.append('Proximos')
    


    est['Estado'] = estados_proy
    est['Links'] = links

    indexNames = est[ est['Links'].str.startswith('https://www.conicyt.cl/becasconicyt') ].index
    # Delete these row indexes from dataFrame
    est.drop(indexNames, inplace=True)



    return est

urls_proyectos = links_proyectos(estados)
urls_proyectos

In [196]:
split_url_proyectos = []
for i in range(0, len(urls_proyectos['Links']), 10):
    split_url_proyectos.append(urls_proyectos['Links'][i:i+10]) 
len(split_url_proyectos)
split_url_proyectos[82]

IndexError: list index out of range

In [201]:
### avance chile
chile = pd.DataFrame()

n = 34
while True:
    chile_sub = pd.DataFrame()
    descripciones = []
    publicos = []
    bitacoras = []
    links = []
    titulos = []

    descripcion = ''
    p_objetivo = ''
    bitacora = ''
    titulo = ''
    link = ''
    print(str(n) + ' ' + 'de 81')
    
    for url_proy in split_url_proyectos[n]:
        #print(url_proy)
        driver = webdriver.Firefox()
        driver.get(url_proy)
        
        #Saca Titulo
                
        try:
            titulo = driver.find_element_by_xpath('.//h2[@id="title-concurso"]').text
            titulos.append(titulo)    
        except:
            try:
                titulo = driver.find_element_by_xpath('.//h2[@class="title_content"]').text
                titulos.append(titulo)
            except:
                titulo = ''
                titulos.append(titulo) 
        
        #Saca descripción

        try:
            boton_descripcion = driver.find_element_by_xpath('//div[contains(text(), "presentación")]')
                    
        except:
            try:
                boton_descripcion = driver.find_element_by_xpath('//li[contains(text(), "Presentación")]')
            except:
                try:
                    boton_descripcion = driver.find_element_by_xpath('//div[contains(@class, "accordion-Presentacin")]')
                except:
                    descripcion = ''
                    descripciones.append(descripcion)
                    
        # ---                
        try:              
            boton_descripcion.click()
            try:
                descripcion = driver.find_element_by_xpath('//div[@id="content-1"]').text
                descripciones.append(descripcion)
            except:
                try:
                    descripcion = driver.find_element_by_xpath('//div[@id="pane-0-0"]').text
                    descripciones.append(descripcion)
                except:
                    try:
                        descripcion = driver.find_element_by_xpath('//div[@class="arconix-accordion-content"]').text
                        descripciones.append(descripcion)
                    except:
                        descripcion = ''
                        descripciones.append(descripcion)
        except:
            descripcion = ''

        sleep(random.randrange(1, 3))

        ### Saca el publico objetivo
        
        try:
            boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "público")]')   
        except:
            try:
                boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "Público")]')
            except:
                try:
                    boton_p_objetivo = driver.find_element_by_xpath('//ul//li[contains(text(), "Público")]') 
                except:
                    p_objetivo = ''
                   
        #---            
        try:                          
            boton_p_objetivo.click()
            try:
                p_objetivo = driver.find_element_by_xpath('.//div[@id="content-2"]').text
                publicos.append(p_objetivo)
            except:
                try:
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-1"]').text    
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-2"]').text    
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-3"]').text
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-4"]').text
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-5"]').text
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-6"]').text
                    
                    publicos.append(p_objetivo)
                except:
                    p_objetivo = ''
                    publicos.append(p_objetivo)
        except:
            p_objetivo = ''
            publicos.append(p_objetivo)

        #print(p_objetivo)
                
        sleep(random.randrange(1, 3))
            
        ### Saca Bitacoras
        try:
            boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "bitacora")]')
        except:
            try:
                boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "bitácora")]')
            except:
                try:
                    boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "Bitácora")]')
                except:
                    try:
                        boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "Bitacora")]')
                    except:
                        try:
                            boton_bitacora = driver.find_element_by_xpath('//li[contains(text(), "bitacora")]') 
                        except:
                            try:
                                boton_bitacora = driver.find_element_by_xpath('//li[contains(text(), "Bitácora")]')
                            except:
                                try:
                                    boton_bitacora = driver.find_element_by_xpath('//li[contains(text(), "Bitacora")]')
                                except:
                                    bitacora = ''
                                    bitacoras.append(bitacora)
                            
        #---                                        
        try:
            boton_bitacora.click()    
            try:
                bitacora = driver.find_element_by_xpath('.//div[@id="content-3"]').text
                bitacoras.append(bitacora)
            except:
                    
                try:
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-1"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-2"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-3"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-4"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-5"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-6"]').text
                    bitacoras.append(bitacora)
                except:
                    bitacora = ''
                    bitacoras.append(bitacora)
        except:
            bitacora = ''
        
        sleep(random.randrange(1, 3))

        ### Extrae los links
        try:
            url = driver.find_elements_by_xpath('//div[@class="collapse show"]/div/ul//li/a')

            urls = []
            for r in url:
                url2 = r.get_attribute('href')
                urls.append(url2)

            if len(urls) == 0:
                url = driver.find_elements_by_xpath('//div[@class="filtros"]/ul//li/a')

                urls = []
                for r in url:
                    url2 = r.get_attribute('href')
                    urls.append(url2)

            urls2 = ''
            for url2 in urls:
                urls2 = urls2 + ', ' + url2
            links.append(urls2)
        except:
            urls2 = ''
            links.append(urls2)


        driver.quit()
        
        #print(len(titulos))
        #print(len(descripciones))
        #print(len(publicos))
        #print(len(bitacoras))
        
        #print(len(links))
        #print(len(split_url_proyectos[n]))
  
        sleep(random.randrange(3, 6))


    if len(titulos)==len(split_url_proyectos[n]) and len(descripciones)==len(split_url_proyectos[n]) and len(publicos)==len(split_url_proyectos[n]) and len(bitacoras)==len(split_url_proyectos[n]) and len(links)==len(split_url_proyectos[n]):
        chile_sub['Título']= titulos
        chile_sub['Descripción']=descripciones
        chile_sub['Público Objetivo']=publicos
        chile_sub['Bitacora']=bitacoras
        chile_sub['Pdfs']= links

        chile = chile.append(chile_sub, ignore_index=True)
        n+=1

    else:
        n=n

    #print(chile)
    
    if n == 81:
        break

34 de 81
35 de 81
36 de 81
37 de 81
38 de 81
39 de 81
40 de 81
41 de 81
42 de 81
43 de 81
44 de 81
45 de 81
46 de 81
47 de 81
48 de 81
49 de 81
49 de 81
49 de 81
49 de 81


KeyboardInterrupt: 

''

In [94]:

#chile_parcial = urls_proyectos[0:len(chile)]

#chile_parcial.reset_index(inplace=True, drop=True)

#chile_final = pd.concat([chile, chile_parcial], axis=1)

#chile_final.to_excel('chile_' + str(n) + '.xlsx')

#chile_final

150
190


In [202]:
chile_final2 = pd.read_excel('chile_34.xlsx')
chile_final2.drop(['Unnamed: 0'], axis=1, inplace=True)
####---------

urls_proyectos.reset_index(inplace=True, drop=True)
urls2 = urls_proyectos[len(chile_final2):len(chile_final2)+len(chile)]
urls2.reset_index(inplace=True, drop=True)
####---------
chile2 = pd.concat([chile, urls2], axis=1)
####---------
chile_final2 = chile_final2.append(chile2, ignore_index=True)
chile_final2.to_excel('chile_' + str(n) + '.xlsx')


In [199]:
driver = webdriver.Firefox()
driver.get('https://www.conicyt.cl/informacioncientifica/2013/08/09/concurso-fondo-de-publicacion-de-revistas-cientificas-chilenas-2013/')
p_objetivo = ''
publicos = []
try:
    boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "público")]')   
except:
    try:
        boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "Público")]')
    except:
        try:
            boton_p_objetivo = driver.find_element_by_xpath('//ul//li[contains(text(), "Público")]') 
        except:
            p_objetivo = ''
            print(5)
            
        #---            
try:                          
    boton_p_objetivo.click()
    try:
        p_objetivo = driver.find_element_by_xpath('.//div[@id="content-2"]').text
        publicos.append(p_objetivo)
        print('4')
    except:
        try:
            if len(p_objetivo) == 0:
                p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-1"]').text    
            if len(p_objetivo) == 0:
                p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-2"]').text    
            if len(p_objetivo) == 0:
                p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-3"]').text
            if len(p_objetivo) == 0:
                p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-4"]').text
            if len(p_objetivo) == 0:
                p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-5"]').text
            if len(p_objetivo) == 0:
                p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-6"]').text
            publicos.append(p_objetivo)
            print('3')
        except:
            p_objetivo = ''
            publicos.append(p_objetivo)
            print('1')
            
except:
    p_objetivo = ''
    print('2')
    publicos.append(p_objetivo)

publicos

2


['']

In [183]:
chile_final2['Links'][0:20].tolist()

['https://www.conicyt.cl/informacioncientifica/2017/06/13/convocatoria-para-indexacon-de-revistas-cientificas-en-scielo-chile/',
 'https://www.conicyt.cl/fondef/2019/11/20/i-concurso-idea-en-dos-etapas-tematico-en-inocuidad-y-calidad-alimentaria-segunda-etapa/',
 'https://www.conicyt.cl/fondef/2019/11/20/iv-concurso-idea-en-dos-etapas-segunda-etapa/',
 'https://www.conicyt.cl/fondef/2019/12/10/ii-concurso-idea-en-dos-etapas-tematico-en-adulto-mayor-segunda-etapa/',
 'https://www.conicyt.cl/pci/2019/12/climat-amsud-convocatoria-internacional-2020/',
 'https://www.conicyt.cl/pci/2019/12/stic-amsud-convocatoria-internacional-2020/',
 'https://www.conicyt.cl/pci/2019/12/math-amsud-convocatoria-internacional-2020/',
 'https://www.conicyt.cl/fondef/2018/08/01/segunda-etapa-proyectos-idea-2016/',
 'https://www.conicyt.cl/fondef/2018/11/22/segunda-etapa-i-concurso-idea-tematico-en-adulto-mayor/',
 'https://www.conicyt.cl/pia/2019/10/03/anillos-de-investigacion-en-ciencia-y-tecnologia-para-la-r

In [203]:
chile_final2

,Título,Descripción,Público Objetivo,Bitacora,Pdfs,Estado,Links
0,CONVOCATORIA PARA INDEXACIÓN DE REVISTAS CIENT...,La convocatoria tiene como objetivo identifica...,Revistas científicas chilenas editadas en form...,Fecha de inicio Postulaciones: 13 de junio de ...,NaN,Abierto,https://www.conicyt.cl/informacioncientifica/2...
1,I Concurso IDeA en dos etapas temático en Inoc...,IMPORTANTE\nSólo pueden postular aquellas inic...,Son entidades elegibles para recibir subsidios...,Fecha de inicio: 25 de septiembre 2019\nFecha ...,", https://www.conicyt.cl/fondef/files/2019/11/...",Abierto,https://www.conicyt.cl/fondef/2019/11/20/i-con...
2,IV Concurso IDeA en Dos Etapas – Segunda Etapa,IMPORTANTE\nSólo pueden postular aquellas inic...,Son entidades elegibles para recibir subsidios...,-Fecha de inicio del concurso: 20 de noviembre...,", https://s3.amazonaws.com/documentos.anid.cl/...",Abierto,https://www.conicyt.cl/fondef/2019/11/20/iv-co...
3,II Concurso IDeA en Dos Etapas temático en Adu...,IMPORTANTE\nATENCIÓN: Sólo pueden postular aqu...,Son entidades elegibles para recibir subsidios...,"Fecha de inicio: lunes 09 de diciembre 2019, 1...",", https://www.conicyt.cl/fondef/files/2017/03/...",Abierto,https://www.conicyt.cl/fondef/2019/12/10/ii-co...
4,"Concurso Programa Regional CLIMAT-Amsud, Año 2020",Objetivos del Concurso:\nDesarrollar la colabo...,"Investigadores/as de universidades, unidades y...",Fecha de inicio de Concurso: 1 de Abril 2020\n...,", https://s3.amazonaws.com/documentos.anid.cl/...",En evaluación,https://www.conicyt.cl/pci/2019/12/climat-amsu...
...,...,...,...,...,...,...,...
485,PREMIO DE EXCELENCIA CIENTÍFICA “ABATE JUAN IG...,El objetivo del “Premio de Excelencia Científi...,"• Investigador/a alemán/a, altamente calificad...",• Fecha de inicio de Concurso: 12 de Abril 201...,,Cerrados,https://www.conicyt.cl/pci/2011/04/premio-de-e...
486,CONCURSO PROPUESTAS DE OBSERVACIÓN ASTRONÓMICA...,Fortalecer la investigación y potenciar la pre...,"• Investigador/a alemán/a, altamente calificad...",• Fecha de inicio de Concurso: 12 de Abril 201...,", https://www.conicyt.cl/astronomia/files/2011...",Cerrados,https://www.conicyt.cl/astronomia/2011/04/11/c...
487,CONCURSO DEL PROGRAMA REGIONAL TICS-AMSUD – CO...,Este concurso est orientado a:\n· Desarrollar ...,"• Investigador/a alemán/a, altamente calificad...",• Fecha de inicio de Concurso: 12 de Abril 201...,,Cerrados,https://www.conicyt.cl/pci/2010/12/concurso-de...
488,CONCURSO PROGRAMA REGIONAL MATH-AMSUD – CONVOC...,Este concurso esta orientad a:\nDesarrollar la...,"• Investigador/a alemán/a, altamente calificad...",• Fecha de inicio de Concurso: 12 de Abril 201...,,Cerrados,https://www.conicyt.cl/pci/2010/12/concurso-pr...
